In [1]:
!nvidia-smi

Thu May 20 02:47:22 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P40           On   | 00000000:00:05.0 Off |                  Off |
| N/A   44C    P0    60W / 250W |      0MiB / 24451MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.utils import collect_env
import os.path as osp
import torch.distributed as dist
from mmcv.runner import load_checkpoint
import time
from mmcv.parallel import MMDataParallel
#from swin_transformer.apex.apex import apex

apex is not installed
apex is not installed
apex is not installed
apex is not installed


In [3]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('/opt/ml/code/swin_transformer/configs/swin/cascade_mask_rcnn_swin_tiny_patch4_window7_mstrain_480-800_giou_4conv1f_adamw_3x_coco.py')
#cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')
PREFIX = '/opt/ml/input/data/'
epoch = 480

from sklearn.model_selection import StratifiedKFold

# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train.json'
cfg.data.train.seg_prefix=PREFIX
#cfg.data.train.pipeline[2]['img_scale'] = (512, 512)

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'
cfg.data.train.seg_prefix=PREFIX
#cfg.data.val.pipeline[1]['img_scale'] = (512, 512)

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
cfg.data.train.seg_prefix=PREFIX
#cfg.data.test.pipeline[1]['img_scale'] = (512, 512)

PREFIX = '../../input/data/'

cfg.data.samples_per_gpu = 4

cfg.seed=2020
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/faster_rcnn_r50_fpn_1x_trash'
cfg.model.roi_head.bbox_head[0].num_classes=11
cfg.model.roi_head.bbox_head[1].num_classes=11
cfg.model.roi_head.bbox_head[2].num_classes=11


# #inferance 노트북
# for i, out in enumerate(output):
#     prediction_string = ''
#     image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
#     for j in range(class_num):
#         for o in out[0][j]:  #삼중 포문에서 마스크 쓴 모델은 이렇게 바꿔줍니다.  
# #cfg.model.roi_head.bbox_head.num_classes = 11
# #cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)

In [4]:
import os
epoch = 1
model = build_detector(cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
checkpoint_path = os.path.join(cfg.work_dir, f'epoch_{epoch}.pth')
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')

Use load_from_local loader


In [5]:
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=3.65s)
creating index...
index created!


/opt/conda/lib/python3.7/site-packages/albumentations/augmentations/transforms.py:2611: UserWarning: blur_limit and sigma_limit minimum value can not be both equal to 0. blur_limit minimum value changed to 3.
  "blur_limit and sigma_limit minimum value can not be both equal to 0. "


In [ ]:
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

loading annotations into memory...


2021-05-20 02:47:52,452 - mmdet - INFO - load checkpoint from /opt/ml/code/cascade_mask_rcnn_swin_tiny_patch4_window7.pth
2021-05-20 02:47:52,453 - mmdet - INFO - Use load_from_local loader


Done (t=1.32s)
creating index...
index created!


2021-05-20 02:47:52,736 - mmdet - WARNING - The model and loaded state dict do not match exactly

size mismatch for rpn_head.rpn_cls.weight: copying a param with shape torch.Size([3, 256, 1, 1]) from checkpoint, the shape in current model is torch.Size([21, 256, 1, 1]).
size mismatch for rpn_head.rpn_cls.bias: copying a param with shape torch.Size([3]) from checkpoint, the shape in current model is torch.Size([21]).
size mismatch for rpn_head.rpn_reg.weight: copying a param with shape torch.Size([12, 256, 1, 1]) from checkpoint, the shape in current model is torch.Size([84, 256, 1, 1]).
size mismatch for rpn_head.rpn_reg.bias: copying a param with shape torch.Size([12]) from checkpoint, the shape in current model is torch.Size([84]).
size mismatch for roi_head.bbox_head.0.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([12, 1024]).
size mismatch for roi_head.bbox_head.0.fc_cls.bias: copying a param with shape to

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 655/655, 2.8 task/s, elapsed: 230s, ETA:     0s

...

2021-05-20 14:15:51,559 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=0.76s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=7.77s).
Accumulating evaluation results...


2021-05-20 14:16:01,868 - mmdet - INFO - Epoch(val) [19][655]	bbox_mAP: 0.2730, bbox_mAP_50: 0.4090, bbox_mAP_75: 0.2780, bbox_mAP_s: 0.0690, bbox_mAP_m: 0.2490, bbox_mAP_l: 0.5060, bbox_mAP_copypaste: 0.273 0.409 0.278 0.069 0.249 0.506


DONE (t=1.60s).
Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.273
Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.409
Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.278
Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.069
Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.249
Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.506
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.448
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.448
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.448
Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.190
Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.443
Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.693


2021-05-20 14:18:33,807 - mmdet - INFO - Epoch [20][50/655]	lr: 7.723e-05, eta: 21:54:21, time: 3.038, data_time: 0.130, memory: 20669, loss_rpn_cls: 0.0314, loss_rpn_bbox: 0.0211, loss_semantic_seg: 0.1111, s0.loss_cls: 0.3207, s0.acc: 89.6816, s0.loss_bbox: 0.1775, s0.loss_mask: 0.2368, s1.loss_cls: 0.1469, s1.acc: 90.4981, s1.loss_bbox: 0.1511, s1.loss_mask: 0.1019, s2.loss_cls: 0.0686, s2.acc: 91.0820, s2.loss_bbox: 0.0784, s2.loss_mask: 0.0443, loss: 1.4898
2021-05-20 14:20:59,213 - mmdet - INFO - Epoch [20][100/655]	lr: 7.723e-05, eta: 21:51:50, time: 2.908, data_time: 0.050, memory: 20669, loss_rpn_cls: 0.0267, loss_rpn_bbox: 0.0231, loss_semantic_seg: 0.1150, s0.loss_cls: 0.3365, s0.acc: 89.2207, s0.loss_bbox: 0.1768, s0.loss_mask: 0.2438, s1.loss_cls: 0.1564, s1.acc: 89.9537, s1.loss_bbox: 0.1575, s1.loss_mask: 0.1063, s2.loss_cls: 0.0736, s2.acc: 90.4174, s2.loss_bbox: 0.0838, s2.loss_mask: 0.0474, loss: 1.5467
2021-05-20 14:23:25,928 - mmdet - INFO - Epoch [20][150/655]	lr: 